<img src="velocity_network.jpg" width="40%">

<img src="radius_network.jpg" width="40%">

# Import Library

In [1]:
# %load TKInversion.py
################################################
########        IMPORT LIBARIES         ########
################################################
from ParamConfig import *
from PathConfig import *
from LibConfig import *

# Set Velocity Inversion Network

In [2]:
################################################
########             NETWORK            ########
################################################

# Here indicating the GPU you want to use. if you don't have GPU, just leave it.
cuda_available = torch.cuda.is_available()
device         = torch.device("cuda" if cuda_available else "cpu")

net = NetModel_v(in_channels=Inchannels,is_deconv=True,is_batchnorm=True,data_dim=DataDim,label_dim=ModelDim)

if torch.cuda.is_available():
    net.cuda()

# Optimizer we want to use
optimizer = torch.optim.Adam(net.parameters(),lr=LearnRate)

# If ReUse, it will load saved model from premodelfilepath and continue to train
if ReUse:
    print('***************** Loading the pre-trained model *****************')
    print('')
    premodel_file = models_dir + premodelname + '.pkl'
    ##Load generator parameters
    net.load_state_dict(torch.load(premodel_file))
    net.to(device)
    print('Finish downloading:',str(premodel_file))

# Loading Data

In [3]:
################################################
########        LOADING  DATA           ########
################################################
print('***************** Loading Training DataSet *****************')
train_set,label_set,data_dsp_dim,label_dsp_dim,train_radius  = DataLoad_Train(train_size=TrainSize,train_data_dir=train_data_dir, \
                                                                 data_dim=DataDim, \
                                                                 model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                                 label_dsp_blk=label_dsp_blk, \
                                                                 truthfilename=truthfilename,velocity_flag=velocity_flag, \
                                                                 position = position)
# Change data type (numpy --> tensor)
train        = data_utils.TensorDataset(torch.from_numpy(train_set),torch.from_numpy(label_set),torch.from_numpy(train_radius))
train_loader = data_utils.DataLoader(train,batch_size=BatchSize,shuffle=True)

print('***************** Loading Testing DataSet *****************')

test_set,label_set_t,data_dsp_dim_t,label_dsp_dim_t,test_radius = DataLoad_Test(test_size=TestSize,test_data_dir=test_data_dir, \
                                                              data_dim=DataDim, \
                                                              model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                              label_dsp_blk=label_dsp_blk, \
                                                              truthfilename=truthfilename,velocity_flag=velocity_flag, \
                                                              position = position)

test        = data_utils.TensorDataset(torch.from_numpy(test_set),torch.from_numpy(label_set_t),torch.from_numpy(test_radius))
test_loader = data_utils.DataLoader(test,batch_size=TestBatchSize,shuffle=False)

***************** Loading Training DataSet *****************
***************** Loading Testing DataSet *****************


# Train Velocity Inversion Network

In [4]:
################################################
########            TRAINING            ########
################################################

print() 
print('*******************************************') 
print('*******************************************') 
print('           START TRAINING                  ') 
print('*******************************************') 
print('*******************************************') 
print() 


print ('Original data dimention:%s'      %  str(DataDim))
print ('Downsampled data dimention:%s '  %  str(data_dsp_dim))
print ('Original label dimention:%s'     %  str(ModelDim))
print ('Downsampled label dimention:%s'  %  str(label_dsp_dim))
print ('Training size:%d'                %  int(TrainSize))
print ('Traning batch size:%d'           %  int(BatchSize))
print ('Number of epochs:%d'             %  int(Epochs))
print ('Learning rate:%.5f'              %  float(LearnRate))
              
# Initialization
loss1  = 0.0
loss2  = 0.0
step   = np.int(TrainSize/BatchSize)
start  = time.time()
min_loss  = float('inf')

for epoch in range(Epochs): 
    epoch_loss = 0.0
    since      = time.time()
    for i, (images,labels,_) in enumerate(train_loader):        
        iteration  = epoch*step+i+1
        # Set Net with train condition
        net.train()
        
        # Reshape data size
        images = images.view(BatchSize,Inchannels,data_dsp_dim[0],data_dsp_dim[1])
        labels = labels.view(BatchSize, -1)
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradient buffer
        optimizer.zero_grad()     
        
        # Forward prediction
        outputs = net(images,label_dsp_dim)
        
        # Calculate the MSE
        loss    = F.mse_loss(outputs,labels,reduction='sum')/(label_dsp_dim[0]*label_dsp_dim[1]*BatchSize)
        
        if np.isnan(float(loss.item())):
            raise ValueError('loss is nan while training')
            
        epoch_loss += loss.item()    
        # Loss backward propagation    
        loss.backward()
        
        # Optimize
        optimizer.step()
        
        # Print loss
        if iteration % DisplayStep == 0:
            print('Epoch: {}/{}, Iteration: {}/{} --- Training Loss:{:.6f}'.format(epoch+1, \
                                                                               Epochs,iteration, \
                                                                              step*Epochs,loss.item()))        
    with torch.no_grad():
        epoch_loss_test = 0.0
        for j, (images,labels,_) in enumerate(test_loader):
            net.eval()
            images = images.view(TestBatchSize,Inchannels,data_dsp_dim[0],data_dsp_dim[1])
            labels = labels.view(TestBatchSize,-1)
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images,label_dsp_dim)
            loss    = F.mse_loss(outputs,labels,reduction='sum')/(label_dsp_dim[0]*label_dsp_dim[1]*TestBatchSize)
            epoch_loss_test += loss.item()
    # Print loss and consuming time every epoch
    if (epoch+1) % 1 == 0:
        #print ('Epoch [%d/%d], Loss: %.10f' % (epoch+1,Epochs,loss.item()))          
        print('Epoch: {:d} finished ! Train Loss: {:.5f}, Test Loss: {:.5f}'.format(epoch+1,epoch_loss/i,epoch_loss_test/j))
        loss1 = np.append(loss1,epoch_loss/i)
        loss2 = np.append(loss2,epoch_loss_test/j)
        time_elapsed = time.time() - since
        print('Epoch consuming time: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
    # Save net parameters with lowest loss
    if epoch_loss_test < min_loss:
        torch.save(net.state_dict(),models_dir+modelname+'_epoch'+'_best'+'_velocity_'+str(Bubble_num)+'.pkl')
        min_loss = epoch_loss_test
        print ('Trained model saved: %d percent completed'% int((epoch+1)*100/Epochs))
    

# Record the consuming time
time_elapsed = time.time() - start
print('Training complete in {:.0f}m  {:.0f}s' .format(time_elapsed //60 , time_elapsed % 60))

# Save the loss
SaveTrainResults_velocity(loss=loss1,loss_test=loss2,SavePath=results_dir,bubble_num=Bubble_num)


*******************************************
*******************************************
           START TRAINING                  
*******************************************
*******************************************

Original data dimention:[37, 5001]
Downsampled data dimention:(1, 2501) 
Original label dimention:[40, 2500]
Downsampled label dimention:(1, 2500)
Training size:500
Traning batch size:10
Number of epochs:200
Learning rate:0.00500
Epoch: 1/200, Iteration: 2/10000 --- Training Loss:107.223900
Epoch: 1/200, Iteration: 4/10000 --- Training Loss:15.050270
Epoch: 1/200, Iteration: 6/10000 --- Training Loss:5.812725
Epoch: 1/200, Iteration: 8/10000 --- Training Loss:4.412106
Epoch: 1/200, Iteration: 10/10000 --- Training Loss:4.207848
Epoch: 1/200, Iteration: 12/10000 --- Training Loss:0.187778
Epoch: 1/200, Iteration: 14/10000 --- Training Loss:1.923652
Epoch: 1/200, Iteration: 16/10000 --- Training Loss:0.505655
Epoch: 1/200, Iteration: 18/10000 --- Training Loss:2.341868


Epoch: 5/200, Iteration: 248/10000 --- Training Loss:0.011978
Epoch: 5/200, Iteration: 250/10000 --- Training Loss:0.029269
Epoch: 5 finished ! Train Loss: 0.01477, Test Loss: 0.01177
Epoch consuming time: 0m 1s
Trained model saved: 2 percent completed
Epoch: 6/200, Iteration: 252/10000 --- Training Loss:0.007527
Epoch: 6/200, Iteration: 254/10000 --- Training Loss:0.007901
Epoch: 6/200, Iteration: 256/10000 --- Training Loss:0.011368
Epoch: 6/200, Iteration: 258/10000 --- Training Loss:0.014096
Epoch: 6/200, Iteration: 260/10000 --- Training Loss:0.006134
Epoch: 6/200, Iteration: 262/10000 --- Training Loss:0.012918
Epoch: 6/200, Iteration: 264/10000 --- Training Loss:0.009460
Epoch: 6/200, Iteration: 266/10000 --- Training Loss:0.063768
Epoch: 6/200, Iteration: 268/10000 --- Training Loss:0.044405
Epoch: 6/200, Iteration: 270/10000 --- Training Loss:0.100174
Epoch: 6/200, Iteration: 272/10000 --- Training Loss:0.047015
Epoch: 6/200, Iteration: 274/10000 --- Training Loss:0.078694
Epo

Epoch: 11/200, Iteration: 504/10000 --- Training Loss:0.008037
Epoch: 11/200, Iteration: 506/10000 --- Training Loss:0.009540
Epoch: 11/200, Iteration: 508/10000 --- Training Loss:0.010215
Epoch: 11/200, Iteration: 510/10000 --- Training Loss:0.024504
Epoch: 11/200, Iteration: 512/10000 --- Training Loss:0.005805
Epoch: 11/200, Iteration: 514/10000 --- Training Loss:0.006163
Epoch: 11/200, Iteration: 516/10000 --- Training Loss:0.006090
Epoch: 11/200, Iteration: 518/10000 --- Training Loss:0.011341
Epoch: 11/200, Iteration: 520/10000 --- Training Loss:0.008731
Epoch: 11/200, Iteration: 522/10000 --- Training Loss:0.008673
Epoch: 11/200, Iteration: 524/10000 --- Training Loss:0.007020
Epoch: 11/200, Iteration: 526/10000 --- Training Loss:0.015808
Epoch: 11/200, Iteration: 528/10000 --- Training Loss:0.011150
Epoch: 11/200, Iteration: 530/10000 --- Training Loss:0.005774
Epoch: 11/200, Iteration: 532/10000 --- Training Loss:0.004733
Epoch: 11/200, Iteration: 534/10000 --- Training Loss:0

Trained model saved: 7 percent completed
Epoch: 16/200, Iteration: 752/10000 --- Training Loss:0.008461
Epoch: 16/200, Iteration: 754/10000 --- Training Loss:0.004278
Epoch: 16/200, Iteration: 756/10000 --- Training Loss:0.004628
Epoch: 16/200, Iteration: 758/10000 --- Training Loss:0.004539
Epoch: 16/200, Iteration: 760/10000 --- Training Loss:0.007682
Epoch: 16/200, Iteration: 762/10000 --- Training Loss:0.002863
Epoch: 16/200, Iteration: 764/10000 --- Training Loss:0.005426
Epoch: 16/200, Iteration: 766/10000 --- Training Loss:0.007876
Epoch: 16/200, Iteration: 768/10000 --- Training Loss:0.004795
Epoch: 16/200, Iteration: 770/10000 --- Training Loss:0.007667
Epoch: 16/200, Iteration: 772/10000 --- Training Loss:0.007658
Epoch: 16/200, Iteration: 774/10000 --- Training Loss:0.005318
Epoch: 16/200, Iteration: 776/10000 --- Training Loss:0.004135
Epoch: 16/200, Iteration: 778/10000 --- Training Loss:0.004438
Epoch: 16/200, Iteration: 780/10000 --- Training Loss:0.004673
Epoch: 16/200,

Epoch: 21/200, Iteration: 1004/10000 --- Training Loss:0.003695
Epoch: 21/200, Iteration: 1006/10000 --- Training Loss:0.003369
Epoch: 21/200, Iteration: 1008/10000 --- Training Loss:0.001887
Epoch: 21/200, Iteration: 1010/10000 --- Training Loss:0.004951
Epoch: 21/200, Iteration: 1012/10000 --- Training Loss:0.003908
Epoch: 21/200, Iteration: 1014/10000 --- Training Loss:0.004137
Epoch: 21/200, Iteration: 1016/10000 --- Training Loss:0.004028
Epoch: 21/200, Iteration: 1018/10000 --- Training Loss:0.003674
Epoch: 21/200, Iteration: 1020/10000 --- Training Loss:0.003974
Epoch: 21/200, Iteration: 1022/10000 --- Training Loss:0.003442
Epoch: 21/200, Iteration: 1024/10000 --- Training Loss:0.003288
Epoch: 21/200, Iteration: 1026/10000 --- Training Loss:0.004021
Epoch: 21/200, Iteration: 1028/10000 --- Training Loss:0.010494
Epoch: 21/200, Iteration: 1030/10000 --- Training Loss:0.005784
Epoch: 21/200, Iteration: 1032/10000 --- Training Loss:0.004505
Epoch: 21/200, Iteration: 1034/10000 ---

...

Epoch: 180/200, Iteration: 8956/10000 --- Training Loss:0.000065
Epoch: 180/200, Iteration: 8958/10000 --- Training Loss:0.000055
Epoch: 180/200, Iteration: 8960/10000 --- Training Loss:0.000107
Epoch: 180/200, Iteration: 8962/10000 --- Training Loss:0.000064
Epoch: 180/200, Iteration: 8964/10000 --- Training Loss:0.000076
Epoch: 180/200, Iteration: 8966/10000 --- Training Loss:0.000520
Epoch: 180/200, Iteration: 8968/10000 --- Training Loss:0.000128
Epoch: 180/200, Iteration: 8970/10000 --- Training Loss:0.000153
Epoch: 180/200, Iteration: 8972/10000 --- Training Loss:0.000075
Epoch: 180/200, Iteration: 8974/10000 --- Training Loss:0.000114
Epoch: 180/200, Iteration: 8976/10000 --- Training Loss:0.000098
Epoch: 180/200, Iteration: 8978/10000 --- Training Loss:0.000126
Epoch: 180/200, Iteration: 8980/10000 --- Training Loss:0.000092
Epoch: 180/200, Iteration: 8982/10000 --- Training Loss:0.000101
Epoch: 180/200, Iteration: 8984/10000 --- Training Loss:0.000170
Epoch: 180/200, Iteration

Epoch: 185/200, Iteration: 9208/10000 --- Training Loss:0.000185
Epoch: 185/200, Iteration: 9210/10000 --- Training Loss:0.000152
Epoch: 185/200, Iteration: 9212/10000 --- Training Loss:0.000124
Epoch: 185/200, Iteration: 9214/10000 --- Training Loss:0.000096
Epoch: 185/200, Iteration: 9216/10000 --- Training Loss:0.000109
Epoch: 185/200, Iteration: 9218/10000 --- Training Loss:0.000177
Epoch: 185/200, Iteration: 9220/10000 --- Training Loss:0.000285
Epoch: 185/200, Iteration: 9222/10000 --- Training Loss:0.000381
Epoch: 185/200, Iteration: 9224/10000 --- Training Loss:0.000128
Epoch: 185/200, Iteration: 9226/10000 --- Training Loss:0.000257
Epoch: 185/200, Iteration: 9228/10000 --- Training Loss:0.000324
Epoch: 185/200, Iteration: 9230/10000 --- Training Loss:0.000183
Epoch: 185/200, Iteration: 9232/10000 --- Training Loss:0.000241
Epoch: 185/200, Iteration: 9234/10000 --- Training Loss:0.000150
Epoch: 185/200, Iteration: 9236/10000 --- Training Loss:0.000193
Epoch: 185/200, Iteration

Epoch: 190/200, Iteration: 9460/10000 --- Training Loss:0.000078
Epoch: 190/200, Iteration: 9462/10000 --- Training Loss:0.000106
Epoch: 190/200, Iteration: 9464/10000 --- Training Loss:0.000131
Epoch: 190/200, Iteration: 9466/10000 --- Training Loss:0.000066
Epoch: 190/200, Iteration: 9468/10000 --- Training Loss:0.000086
Epoch: 190/200, Iteration: 9470/10000 --- Training Loss:0.000056
Epoch: 190/200, Iteration: 9472/10000 --- Training Loss:0.000087
Epoch: 190/200, Iteration: 9474/10000 --- Training Loss:0.000068
Epoch: 190/200, Iteration: 9476/10000 --- Training Loss:0.000186
Epoch: 190/200, Iteration: 9478/10000 --- Training Loss:0.000072
Epoch: 190/200, Iteration: 9480/10000 --- Training Loss:0.000106
Epoch: 190/200, Iteration: 9482/10000 --- Training Loss:0.000087
Epoch: 190/200, Iteration: 9484/10000 --- Training Loss:0.000065
Epoch: 190/200, Iteration: 9486/10000 --- Training Loss:0.000100
Epoch: 190/200, Iteration: 9488/10000 --- Training Loss:0.000127
Epoch: 190/200, Iteration

Epoch: 195/200, Iteration: 9712/10000 --- Training Loss:0.000097
Epoch: 195/200, Iteration: 9714/10000 --- Training Loss:0.000257
Epoch: 195/200, Iteration: 9716/10000 --- Training Loss:0.000145
Epoch: 195/200, Iteration: 9718/10000 --- Training Loss:0.000173
Epoch: 195/200, Iteration: 9720/10000 --- Training Loss:0.000125
Epoch: 195/200, Iteration: 9722/10000 --- Training Loss:0.000121
Epoch: 195/200, Iteration: 9724/10000 --- Training Loss:0.000102
Epoch: 195/200, Iteration: 9726/10000 --- Training Loss:0.000105
Epoch: 195/200, Iteration: 9728/10000 --- Training Loss:0.000065
Epoch: 195/200, Iteration: 9730/10000 --- Training Loss:0.000097
Epoch: 195/200, Iteration: 9732/10000 --- Training Loss:0.000138
Epoch: 195/200, Iteration: 9734/10000 --- Training Loss:0.000060
Epoch: 195/200, Iteration: 9736/10000 --- Training Loss:0.000131
Epoch: 195/200, Iteration: 9738/10000 --- Training Loss:0.000062
Epoch: 195/200, Iteration: 9740/10000 --- Training Loss:0.000087
Epoch: 195/200, Iteration

Epoch: 200/200, Iteration: 9952/10000 --- Training Loss:0.000113
Epoch: 200/200, Iteration: 9954/10000 --- Training Loss:0.000146
Epoch: 200/200, Iteration: 9956/10000 --- Training Loss:0.000133
Epoch: 200/200, Iteration: 9958/10000 --- Training Loss:0.000183
Epoch: 200/200, Iteration: 9960/10000 --- Training Loss:0.000123
Epoch: 200/200, Iteration: 9962/10000 --- Training Loss:0.000075
Epoch: 200/200, Iteration: 9964/10000 --- Training Loss:0.000077
Epoch: 200/200, Iteration: 9966/10000 --- Training Loss:0.000115
Epoch: 200/200, Iteration: 9968/10000 --- Training Loss:0.000087
Epoch: 200/200, Iteration: 9970/10000 --- Training Loss:0.000048
Epoch: 200/200, Iteration: 9972/10000 --- Training Loss:0.000131
Epoch: 200/200, Iteration: 9974/10000 --- Training Loss:0.000144
Epoch: 200/200, Iteration: 9976/10000 --- Training Loss:0.000114
Epoch: 200/200, Iteration: 9978/10000 --- Training Loss:0.000118
Epoch: 200/200, Iteration: 9980/10000 --- Training Loss:0.000060
Epoch: 200/200, Iteration

# Load the Trained Velocity Inversion Network with Lowest Loss

In [5]:
#velocity
if ReUse_best:
    print('***************** Loading the best trained model *****************')
    print('')
    cuda_available = torch.cuda.is_available()
    device         = torch.device("cuda" if cuda_available else "cpu")
    net = NetModel_v(in_channels=Inchannels,is_deconv=True,is_batchnorm=True,data_dim=DataDim,label_dim=ModelDim)
    premodel_file = models_dir+modelname+'_epoch'+'_best'+'_velocity_'+str(Bubble_num)+'.pkl'
    if torch.cuda.is_available():
        net.cuda()
    ##Load generator parameters
    net.load_state_dict(torch.load(premodel_file))
    net.to(device)
    print('Finish downloading:',str(premodel_file))
    
    print('***************** Loading Training DataSet *****************')
    train_set,label_set,data_dsp_dim,label_dsp_dim,train_radius  = DataLoad_Train(train_size=TrainSize,train_data_dir=train_data_dir, \
                                                                 data_dim=DataDim, \
                                                                 model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                                 label_dsp_blk=label_dsp_blk, \
                                                                 truthfilename=truthfilename,velocity_flag=velocity_flag, \
                                                                 position = position)
    # Change data type (numpy --> tensor)
    train        = data_utils.TensorDataset(torch.from_numpy(train_set),torch.from_numpy(label_set),torch.from_numpy(train_radius))
    train_loader = data_utils.DataLoader(train,batch_size=BatchSize,shuffle=True)

    print('***************** Loading Testing DataSet  *****************')

    test_set,label_set_t,data_dsp_dim_t,label_dsp_dim_t,test_radius = DataLoad_Test(test_size=TestSize,test_data_dir=test_data_dir, \
                                                              data_dim=DataDim, \
                                                              model_dim=ModelDim,data_dsp_blk=data_dsp_blk, \
                                                              label_dsp_blk=label_dsp_blk, \
                                                              truthfilename=truthfilename,velocity_flag=velocity_flag, \
                                                              position = position)

    test        = data_utils.TensorDataset(torch.from_numpy(test_set),torch.from_numpy(label_set_t),torch.from_numpy(test_radius))
    test_loader = data_utils.DataLoader(test,batch_size=TestBatchSize,shuffle=False)
    
    print('***************** Loading Dataset Success! *****************')

***************** Loading the best trained model *****************

Finish downloading: /home/pingchuan/TokyoBayInversion_2022_3_14/models/SimulataModel/Simulate_FCNVMBModel_TrainSize500_Epoch200_BatchSize10_LR0.005_epoch_best_velocity_1.pkl
***************** Loading Training DataSet *****************
***************** Loading Testing DataSet  *****************
***************** Loading Dataset Success! *****************


# Predicting Velocity From Waveforms by Velocity Inversion Network

In [6]:
#Test velocity
################################################
########            TESTING             ########
################################################

print() 
print('*******************************************') 
print('*******************************************') 
print('            START TESTING                  ') 
print('*******************************************') 
print('*******************************************') 
print()

# Initialization
since            = time.time()
Prediction_test  = np.zeros((TestSize,label_dsp_dim[0],label_dsp_dim[1]),dtype=float)
GT               = np.zeros((TestSize,label_dsp_dim[0],label_dsp_dim[1]),dtype=float)
Prediction_train = np.zeros((TrainSize,label_dsp_dim[0],label_dsp_dim[1]),dtype=float)
radius_train     = np.zeros((TrainSize,Bubble_num),dtype=float)
radius_test      = np.zeros((TestSize,Bubble_num),dtype=float)
total      = 0

#train dataset
for i, (images,_,radius) in enumerate(train_loader):        
    images = images.view(BatchSize,Inchannels,data_dsp_dim[0],data_dsp_dim[1])
    images = images.to(device)
    
    # Predictions
    net.eval() 
    outputs  = net(images,label_dsp_dim)
    outputs  = outputs.view(BatchSize,label_dsp_dim[0],label_dsp_dim[1])
    outputs  = outputs.data.cpu().numpy()
    
    # Store Velocity Prediction and corresponding radius for next Network
    for k in range(BatchSize):
        pd   = outputs[k,:,:].reshape(label_dsp_dim[0],label_dsp_dim[1])
        radius_train[i*BatchSize+k,:] = radius[k].reshape(-1)
        Prediction_train[i*BatchSize+k,:,:] = pd
        total = total + 1

#test dataset
for i, (images,labels,radius) in enumerate(test_loader):        
    images = images.view(TestBatchSize,Inchannels,data_dsp_dim[0],data_dsp_dim[1])
    labels = labels.view(TestBatchSize,Nclasses,label_dsp_dim[0],label_dsp_dim[1])
    images = images.to(device)
    labels = labels.to(device)
    
    # Predictions
    net.eval() 
    outputs  = net(images,label_dsp_dim)
    outputs  = outputs.view(TestBatchSize,label_dsp_dim[0],label_dsp_dim[1])
    outputs  = outputs.data.cpu().numpy()
    gts      = labels.data.cpu().numpy()
    
    # Store Velocity Prediction and corresponding radius for next Network
    for k in range(TestBatchSize):
        pd   = outputs[k,:,:].reshape(label_dsp_dim[0],label_dsp_dim[1])
        gt   = gts[k,:,:].reshape(label_dsp_dim[0],label_dsp_dim[1])
        Prediction_test[i*TestBatchSize+k,:,:] = pd
        GT[i*TestBatchSize+k,:,:] = gt
        radius_test[i*TestBatchSize+k,:] = radius[k].reshape(-1)
        total = total + 1

# Save Results
SaveTestResults_velocity(Prediction_test,GT,radius_test,results_dir,Bubble_num)
        
# Record the consuming time
time_elapsed = time.time() - since
print('Testing complete in  {:.0f}m {:.0f}s' .format(time_elapsed // 60, time_elapsed % 60))


*******************************************
*******************************************
            START TESTING                  
*******************************************
*******************************************

Testing complete in  0m 0s


# Train Radius Inversion Network

In [7]:
#predict radius
cuda_available = torch.cuda.is_available()
device         = torch.device("cuda" if cuda_available else "cpu")
net_r = NetModel_r(in_channels=Inchannels,is_deconv=True,is_batchnorm=True,data_dim=label_dsp_dim,bubble_num=Bubble_num)
if torch.cuda.is_available():
    net_r.cuda()

# Optimizer we want to use
optimizer = torch.optim.Adam(net_r.parameters(),lr=LearnRate)

train_r        = data_utils.TensorDataset(torch.from_numpy(Prediction_train),torch.from_numpy(radius_train))
train_loader_r = data_utils.DataLoader(train_r,batch_size=BatchSize,shuffle=True)
test_r         = data_utils.TensorDataset(torch.from_numpy(Prediction_test),torch.from_numpy(radius_test))
test_loader_r  = data_utils.DataLoader(test_r,batch_size=TestBatchSize,shuffle=False)

# Initialization
loss1  = 0.0
loss2  = 0.0
step   = np.int(TrainSize/BatchSize)
start  = time.time()
min_loss  = float('inf')

for epoch in range(Epochs): 
    epoch_loss = 0.0
    since      = time.time()
    for i, (images,labels) in enumerate(train_loader_r):        
        iteration  = epoch*step+i+1
        # Set Net with train condition
        net_r.train()
        
        # Reshape data size
        images = images.view(BatchSize,-1).float()
        labels = labels.view(BatchSize, -1).float()
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradient buffer
        optimizer.zero_grad()     
        
        # Forward prediction
        outputs = net_r(images)
        
        # Calculate the MSE
        loss    = F.mse_loss(outputs,labels,reduction='sum')/(Bubble_num*BatchSize)
        
        if np.isnan(float(loss.item())):
            raise ValueError('loss is nan while training')
            
        epoch_loss += loss.item()    
        # Loss backward propagation    
        loss.backward()
        
        # Optimize
        optimizer.step()
        
        # Print loss
        if iteration % DisplayStep == 0:
            print('Epoch: {}/{}, Iteration: {}/{} --- Training Loss:{:.6f}'.format(epoch+1, \
                                                                               Epochs,iteration, \
                                                                              step*Epochs,loss.item()))        
    with torch.no_grad():
        epoch_loss_test = 0.0
        for j, (images,labels) in enumerate(test_loader_r):
            net_r.eval()
            images = images.view(TestBatchSize,-1).float()
            labels = labels.view(TestBatchSize,-1).float()
            images = images.to(device)
            labels = labels.to(device)
            outputs = net_r(images)
            loss    = F.mse_loss(outputs,labels,reduction='sum')/(Bubble_num*TestBatchSize)
            epoch_loss_test += loss.item()
    # Print loss and consuming time every epoch
    if (epoch+1) % 1 == 0:
        #print ('Epoch [%d/%d], Loss: %.10f' % (epoch+1,Epochs,loss.item()))          
        print('Epoch: {:d} finished ! Train Loss: {:.5f}, Test Loss: {:.5f}'.format(epoch+1,epoch_loss/i,epoch_loss_test/j))
        loss1 = np.append(loss1,epoch_loss/i)
        loss2 = np.append(loss2,epoch_loss_test/j)
        time_elapsed = time.time() - since
        print('Epoch consuming time: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
    # Save net_r parameters with lowest loss
    if epoch_loss_test < min_loss:
        torch.save(net_r.state_dict(),models_dir+modelname+'_epoch'+'_best'+'_radius_'+str(Bubble_num)+'.pkl')
        min_loss = epoch_loss_test
        print ('Trained model saved: %d percent completed'% int((epoch+1)*100/Epochs))
    

# Record the consuming time
time_elapsed = time.time() - start
print('Training complete in {:.0f}m  {:.0f}s' .format(time_elapsed //60 , time_elapsed % 60))

# Save the loss
SaveTrainResults_radius(loss=loss1,loss_test=loss2,SavePath=results_dir,bubble_num=Bubble_num)

Epoch: 1/200, Iteration: 2/10000 --- Training Loss:3063.921875
Epoch: 1/200, Iteration: 4/10000 --- Training Loss:1830.165405
Epoch: 1/200, Iteration: 6/10000 --- Training Loss:3112.053711
Epoch: 1/200, Iteration: 8/10000 --- Training Loss:1441.832520
Epoch: 1/200, Iteration: 10/10000 --- Training Loss:580.625916
Epoch: 1/200, Iteration: 12/10000 --- Training Loss:963.934753
Epoch: 1/200, Iteration: 14/10000 --- Training Loss:749.679993
Epoch: 1/200, Iteration: 16/10000 --- Training Loss:1040.624390
Epoch: 1/200, Iteration: 18/10000 --- Training Loss:1134.265259
Epoch: 1/200, Iteration: 20/10000 --- Training Loss:536.356873
Epoch: 1/200, Iteration: 22/10000 --- Training Loss:634.817871
Epoch: 1/200, Iteration: 24/10000 --- Training Loss:815.229919
Epoch: 1/200, Iteration: 26/10000 --- Training Loss:608.982910
Epoch: 1/200, Iteration: 28/10000 --- Training Loss:631.593689
Epoch: 1/200, Iteration: 30/10000 --- Training Loss:719.506470
Epoch: 1/200, Iteration: 32/10000 --- Training Loss:8

Epoch: 6/200, Iteration: 264/10000 --- Training Loss:35.463043
Epoch: 6/200, Iteration: 266/10000 --- Training Loss:53.912769
Epoch: 6/200, Iteration: 268/10000 --- Training Loss:33.667492
Epoch: 6/200, Iteration: 270/10000 --- Training Loss:39.100101
Epoch: 6/200, Iteration: 272/10000 --- Training Loss:103.906090
Epoch: 6/200, Iteration: 274/10000 --- Training Loss:28.505426
Epoch: 6/200, Iteration: 276/10000 --- Training Loss:53.639015
Epoch: 6/200, Iteration: 278/10000 --- Training Loss:65.009125
Epoch: 6/200, Iteration: 280/10000 --- Training Loss:60.239979
Epoch: 6/200, Iteration: 282/10000 --- Training Loss:47.636494
Epoch: 6/200, Iteration: 284/10000 --- Training Loss:17.232407
Epoch: 6/200, Iteration: 286/10000 --- Training Loss:19.999443
Epoch: 6/200, Iteration: 288/10000 --- Training Loss:44.739250
Epoch: 6/200, Iteration: 290/10000 --- Training Loss:55.179127
Epoch: 6/200, Iteration: 292/10000 --- Training Loss:40.503033
Epoch: 6/200, Iteration: 294/10000 --- Training Loss:2

Epoch: 11 finished ! Train Loss: 4.64770, Test Loss: 6.00746
Epoch consuming time: 0m 0s
Epoch: 12/200, Iteration: 552/10000 --- Training Loss:0.307975
Epoch: 12/200, Iteration: 554/10000 --- Training Loss:7.371383
Epoch: 12/200, Iteration: 556/10000 --- Training Loss:1.297691
Epoch: 12/200, Iteration: 558/10000 --- Training Loss:1.029791
Epoch: 12/200, Iteration: 560/10000 --- Training Loss:0.404721
Epoch: 12/200, Iteration: 562/10000 --- Training Loss:10.615885
Epoch: 12/200, Iteration: 564/10000 --- Training Loss:0.677924
Epoch: 12/200, Iteration: 566/10000 --- Training Loss:1.972993
Epoch: 12/200, Iteration: 568/10000 --- Training Loss:9.571351
Epoch: 12/200, Iteration: 570/10000 --- Training Loss:3.061698
Epoch: 12/200, Iteration: 572/10000 --- Training Loss:6.627300
Epoch: 12/200, Iteration: 574/10000 --- Training Loss:0.395823
Epoch: 12/200, Iteration: 576/10000 --- Training Loss:0.463499
Epoch: 12/200, Iteration: 578/10000 --- Training Loss:4.707957
Epoch: 12/200, Iteration: 58

Epoch: 17/200, Iteration: 842/10000 --- Training Loss:0.726558
Epoch: 17/200, Iteration: 844/10000 --- Training Loss:1.009157
Epoch: 17/200, Iteration: 846/10000 --- Training Loss:0.982211
Epoch: 17/200, Iteration: 848/10000 --- Training Loss:2.840250
Epoch: 17/200, Iteration: 850/10000 --- Training Loss:3.350950
Epoch: 17 finished ! Train Loss: 1.65198, Test Loss: 10.60896
Epoch consuming time: 0m 0s
Epoch: 18/200, Iteration: 852/10000 --- Training Loss:0.471154
Epoch: 18/200, Iteration: 854/10000 --- Training Loss:0.728065
Epoch: 18/200, Iteration: 856/10000 --- Training Loss:0.963144
Epoch: 18/200, Iteration: 858/10000 --- Training Loss:1.243032
Epoch: 18/200, Iteration: 860/10000 --- Training Loss:0.819592
Epoch: 18/200, Iteration: 862/10000 --- Training Loss:3.335923
Epoch: 18/200, Iteration: 864/10000 --- Training Loss:1.124298
Epoch: 18/200, Iteration: 866/10000 --- Training Loss:0.492524
Epoch: 18/200, Iteration: 868/10000 --- Training Loss:0.518238
Epoch: 18/200, Iteration: 87

Epoch: 23/200, Iteration: 1130/10000 --- Training Loss:0.373971
Epoch: 23/200, Iteration: 1132/10000 --- Training Loss:0.833696
Epoch: 23/200, Iteration: 1134/10000 --- Training Loss:0.428179
Epoch: 23/200, Iteration: 1136/10000 --- Training Loss:4.340922
Epoch: 23/200, Iteration: 1138/10000 --- Training Loss:0.132194
Epoch: 23/200, Iteration: 1140/10000 --- Training Loss:0.848643
Epoch: 23/200, Iteration: 1142/10000 --- Training Loss:1.562815
Epoch: 23/200, Iteration: 1144/10000 --- Training Loss:1.435887
Epoch: 23/200, Iteration: 1146/10000 --- Training Loss:2.305510
Epoch: 23/200, Iteration: 1148/10000 --- Training Loss:1.285189
Epoch: 23/200, Iteration: 1150/10000 --- Training Loss:0.701215
Epoch: 23 finished ! Train Loss: 1.26298, Test Loss: 10.15078
Epoch consuming time: 0m 0s
Epoch: 24/200, Iteration: 1152/10000 --- Training Loss:4.003129
Epoch: 24/200, Iteration: 1154/10000 --- Training Loss:0.226291
Epoch: 24/200, Iteration: 1156/10000 --- Training Loss:0.655741
Epoch: 24/200,

...

Epoch: 172/200, Iteration: 8600/10000 --- Training Loss:0.913428
Epoch: 172 finished ! Train Loss: 0.58452, Test Loss: 1.46010
Epoch consuming time: 0m 0s
Epoch: 173/200, Iteration: 8602/10000 --- Training Loss:0.577199
Epoch: 173/200, Iteration: 8604/10000 --- Training Loss:0.335191
Epoch: 173/200, Iteration: 8606/10000 --- Training Loss:0.313546
Epoch: 173/200, Iteration: 8608/10000 --- Training Loss:0.363915
Epoch: 173/200, Iteration: 8610/10000 --- Training Loss:0.359890
Epoch: 173/200, Iteration: 8612/10000 --- Training Loss:0.503208
Epoch: 173/200, Iteration: 8614/10000 --- Training Loss:0.603418
Epoch: 173/200, Iteration: 8616/10000 --- Training Loss:0.378586
Epoch: 173/200, Iteration: 8618/10000 --- Training Loss:0.446233
Epoch: 173/200, Iteration: 8620/10000 --- Training Loss:0.537011
Epoch: 173/200, Iteration: 8622/10000 --- Training Loss:0.298865
Epoch: 173/200, Iteration: 8624/10000 --- Training Loss:0.612324
Epoch: 173/200, Iteration: 8626/10000 --- Training Loss:0.842482


Epoch: 178/200, Iteration: 8886/10000 --- Training Loss:0.422565
Epoch: 178/200, Iteration: 8888/10000 --- Training Loss:0.205614
Epoch: 178/200, Iteration: 8890/10000 --- Training Loss:1.045702
Epoch: 178/200, Iteration: 8892/10000 --- Training Loss:0.838344
Epoch: 178/200, Iteration: 8894/10000 --- Training Loss:0.207528
Epoch: 178/200, Iteration: 8896/10000 --- Training Loss:0.542191
Epoch: 178/200, Iteration: 8898/10000 --- Training Loss:0.148508
Epoch: 178/200, Iteration: 8900/10000 --- Training Loss:0.897359
Epoch: 178 finished ! Train Loss: 0.50799, Test Loss: 1.76791
Epoch consuming time: 0m 0s
Epoch: 179/200, Iteration: 8902/10000 --- Training Loss:0.805781
Epoch: 179/200, Iteration: 8904/10000 --- Training Loss:0.574495
Epoch: 179/200, Iteration: 8906/10000 --- Training Loss:0.872093
Epoch: 179/200, Iteration: 8908/10000 --- Training Loss:0.683014
Epoch: 179/200, Iteration: 8910/10000 --- Training Loss:2.119310
Epoch: 179/200, Iteration: 8912/10000 --- Training Loss:0.916790


Epoch: 184/200, Iteration: 9172/10000 --- Training Loss:0.419972
Epoch: 184/200, Iteration: 9174/10000 --- Training Loss:0.444439
Epoch: 184/200, Iteration: 9176/10000 --- Training Loss:0.365061
Epoch: 184/200, Iteration: 9178/10000 --- Training Loss:0.411791
Epoch: 184/200, Iteration: 9180/10000 --- Training Loss:0.456135
Epoch: 184/200, Iteration: 9182/10000 --- Training Loss:0.786725
Epoch: 184/200, Iteration: 9184/10000 --- Training Loss:0.916427
Epoch: 184/200, Iteration: 9186/10000 --- Training Loss:0.214031
Epoch: 184/200, Iteration: 9188/10000 --- Training Loss:0.377959
Epoch: 184/200, Iteration: 9190/10000 --- Training Loss:0.583431
Epoch: 184/200, Iteration: 9192/10000 --- Training Loss:0.851246
Epoch: 184/200, Iteration: 9194/10000 --- Training Loss:0.508140
Epoch: 184/200, Iteration: 9196/10000 --- Training Loss:1.439586
Epoch: 184/200, Iteration: 9198/10000 --- Training Loss:1.169764
Epoch: 184/200, Iteration: 9200/10000 --- Training Loss:0.891981
Epoch: 184 finished ! Tra

Epoch: 190/200, Iteration: 9460/10000 --- Training Loss:0.734176
Epoch: 190/200, Iteration: 9462/10000 --- Training Loss:0.508850
Epoch: 190/200, Iteration: 9464/10000 --- Training Loss:0.333976
Epoch: 190/200, Iteration: 9466/10000 --- Training Loss:0.308136
Epoch: 190/200, Iteration: 9468/10000 --- Training Loss:0.472934
Epoch: 190/200, Iteration: 9470/10000 --- Training Loss:0.332277
Epoch: 190/200, Iteration: 9472/10000 --- Training Loss:0.235010
Epoch: 190/200, Iteration: 9474/10000 --- Training Loss:0.153347
Epoch: 190/200, Iteration: 9476/10000 --- Training Loss:0.871339
Epoch: 190/200, Iteration: 9478/10000 --- Training Loss:0.474098
Epoch: 190/200, Iteration: 9480/10000 --- Training Loss:0.730334
Epoch: 190/200, Iteration: 9482/10000 --- Training Loss:0.658374
Epoch: 190/200, Iteration: 9484/10000 --- Training Loss:0.233004
Epoch: 190/200, Iteration: 9486/10000 --- Training Loss:0.702071
Epoch: 190/200, Iteration: 9488/10000 --- Training Loss:1.219908
Epoch: 190/200, Iteration

Epoch: 195/200, Iteration: 9750/10000 --- Training Loss:0.398403
Epoch: 195 finished ! Train Loss: 0.55729, Test Loss: 2.28898
Epoch consuming time: 0m 0s
Epoch: 196/200, Iteration: 9752/10000 --- Training Loss:0.654224
Epoch: 196/200, Iteration: 9754/10000 --- Training Loss:0.300763
Epoch: 196/200, Iteration: 9756/10000 --- Training Loss:0.487577
Epoch: 196/200, Iteration: 9758/10000 --- Training Loss:0.431493
Epoch: 196/200, Iteration: 9760/10000 --- Training Loss:0.492688
Epoch: 196/200, Iteration: 9762/10000 --- Training Loss:0.121832
Epoch: 196/200, Iteration: 9764/10000 --- Training Loss:0.559517
Epoch: 196/200, Iteration: 9766/10000 --- Training Loss:0.192561
Epoch: 196/200, Iteration: 9768/10000 --- Training Loss:0.414409
Epoch: 196/200, Iteration: 9770/10000 --- Training Loss:0.377698
Epoch: 196/200, Iteration: 9772/10000 --- Training Loss:0.223236
Epoch: 196/200, Iteration: 9774/10000 --- Training Loss:0.230013
Epoch: 196/200, Iteration: 9776/10000 --- Training Loss:0.313920


# Load the Trained Radius Inversion Network with Lowest Loss

In [8]:
#radius
if ReUse_best:
    print('***************** Loading the best trained model *****************')
    print('')
    cuda_available = torch.cuda.is_available()
    device         = torch.device("cuda" if cuda_available else "cpu")
    net_r = NetModel_r(in_channels=Inchannels,is_deconv=True,is_batchnorm=True,data_dim=label_dsp_dim,bubble_num=Bubble_num)
    premodel_file = models_dir+modelname+'_epoch'+'_best'+'_radius_'+str(Bubble_num)+'.pkl'
    if torch.cuda.is_available():
        net_r.cuda()
    ##Load generator parameters
    net_r.load_state_dict(torch.load(premodel_file))
    net_r.to(device)
    print('Finish downloading:',str(premodel_file))

***************** Loading the best trained model *****************

Finish downloading: /home/pingchuan/TokyoBayInversion_2022_3_14/models/SimulataModel/Simulate_FCNVMBModel_TrainSize500_Epoch200_BatchSize10_LR0.005_epoch_best_radius_1.pkl


# Predicting Radius From Predicted Velocity by Radius Inversion Network

In [10]:
#test radius
print() 
print('*******************************************') 
print('*******************************************') 
print('            START TESTING                  ') 
print('*******************************************') 
print('*******************************************') 
print()

# Initialization
since      = time.time()
Prediction = np.zeros((TestSize,Bubble_num),dtype=float)
GT         = np.zeros((TestSize,Bubble_num),dtype=float) #GT = Ground Truth
total      = 0
for i, (images,labels) in enumerate(test_loader_r):        
    images = images.view(TestBatchSize, -1).float()
    labels = labels.view(TestBatchSize, -1).float()
    images = images.to(device)
    labels = labels.to(device)
    
    # Predictions
    net_r.eval() 
    outputs  = net_r(images)
    outputs  = outputs.view(TestBatchSize,-1)
    outputs  = outputs.data.cpu().numpy()
    gts      = labels.data.cpu().numpy()
    
    # Calculate Prediction and GroundTruth
    for k in range(TestBatchSize):
        pd   = outputs[k,:].reshape(-1)
        gt   = gts[k,:].reshape(-1)
        Prediction[i*TestBatchSize+k,:] = pd
        GT[i*TestBatchSize+k,:] = gt
        total = total + 1

# Save Results
SaveTestResults_radius(Prediction,GT,results_dir,Bubble_num)
        
# Record the consuming time
time_elapsed = time.time() - since
print('Testing complete in  {:.0f}m {:.0f}s' .format(time_elapsed // 60, time_elapsed % 60))


*******************************************
*******************************************
            START TESTING                  
*******************************************
*******************************************

Testing complete in  0m 0s
